In [330]:
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow import keras as K, nn


In [331]:
# Discover labeled wav files and load audio
wav_files = sorted(Path('.').glob('*.wav'))
labeled_files = []
for wav_path in wav_files:
    stem = wav_path.stem
    if 'no_presence' in stem:
        label = 0.0
    elif 'presence' in stem:
        label = 1.0
    else:
        continue
    labeled_files.append((stem, wav_path, label))

if not labeled_files:
    raise ValueError("No labeled wav files found (filenames should contain 'presence' or 'no_presence').")

dataset = []
sample_rate = None
for stem, wav_path, label in labeled_files:
    audio, sr = sf.read(wav_path, dtype='float32')
    if sample_rate is None:
        sample_rate = sr
    elif sr != sample_rate:
        raise ValueError(f"Sample rate mismatch for {wav_path}: {sr} vs {sample_rate}")
    dataset.append({'name': stem, 'audio': audio, 'label': label})
    print(f"Loaded {wav_path} label={label} samples={len(audio)} sr={sr}")

y = dataset[0]['audio']
sr = sample_rate


Loaded group_room_2_9_dec_presence.wav label=1.0 samples=1920000 sr=16000
Loaded group_room_3_yazan_lab_9_dec_presence.wav label=1.0 samples=960000 sr=16000
Loaded group_room_4_9_dec_no_presence.wav label=0.0 samples=13312 sr=16000
Loaded group_room_4_9_dec_presence.wav label=1.0 samples=1920256 sr=16000
Loaded group_room_5_9_dec_presence.wav label=1.0 samples=1920000 sr=16000
Loaded group_room_9_dec_presence.wav label=1.0 samples=960000 sr=16000
Loaded test_presence.wav label=1.0 samples=80128 sr=16000
Loaded training-lax_no_presence.wav label=0.0 samples=960000 sr=16000
Loaded training-lax_presence.wav label=1.0 samples=960256 sr=16000
Loaded training2-lax_no_presence.wav label=0.0 samples=960000 sr=16000
Loaded training2-lax_presence.wav label=1.0 samples=960000 sr=16000


In [332]:
y

array([ 0.0647583 ,  0.07305908,  0.07733154, ..., -0.00384521,
        0.00158691,  0.00167847], shape=(1920000,), dtype=float32)

In [333]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        if i + n > len(lst):
            return
        yield lst[i : i + n]


CHUNK_DURATION_S = 0.5  # 0.5s chunks
SEGMENT_DURATION_S = 2.0  # 2s windows (4 chunks)
SEGMENT_HOP_CHUNKS = 1    # slide by 1 chunk -> 0.5s hop


def chunk_fft_features(signal, sample_rate, chunk_duration=CHUNK_DURATION_S):
    """Chunk audio and compute binned FFT magnitudes for each chunk."""
    chunk_size = int(sample_rate * chunk_duration)
    chunked = np.asarray(list(chunks(signal, chunk_size)))
    if chunked.size == 0:
        return np.empty((0, 0), dtype=np.float32)

    fft_mag = np.abs(np.fft.rfft(chunked, axis=1))
    n_chunks, n_bins = fft_mag.shape

    bin_width = sample_rate / chunk_size

    def hz_to_bin(hz: float) -> int:
        return int(np.floor(hz / bin_width))

    bin_320 = min(hz_to_bin(320.0), n_bins)      # 0–320 Hz
    bin_3200 = min(hz_to_bin(3200.0), n_bins)    # 320–3200 Hz

    def merge_region(region: np.ndarray, target_bw_hz: float) -> np.ndarray:
        if region.shape[1] == 0:
            return region[:, :0]

        bins_per = max(1, int(round(target_bw_hz / bin_width)))
        usable = (region.shape[1] // bins_per) * bins_per
        if usable == 0:
            return region[:, :0]

        region = region[:, :usable]
        return region.reshape(n_chunks, -1, bins_per).mean(axis=2)

    low_raw = fft_mag[:, :bin_320]            # ~0–320 Hz
    mid_raw = fft_mag[:, bin_320:bin_3200]    # ~320–3200 Hz
    high_raw = fft_mag[:, bin_3200:]          # >3200 Hz

    low = merge_region(low_raw, 4.0)      # ≈ 4 Hz bins
    mid = merge_region(mid_raw, 32.0)     # ≈ 32 Hz bins
    high = merge_region(high_raw, 128.0)  # ≈ 128 Hz bins

    features = np.concatenate([low, mid, high], axis=1)
    return features.astype(np.float32)


def make_segments(
    feature_array,
    segment_duration=SEGMENT_DURATION_S,
    chunk_duration=CHUNK_DURATION_S,
    hop_chunks=SEGMENT_HOP_CHUNKS,
):
    """Group FFT chunks into overlapping segments (rolling windows)."""
    chunks_per_segment = int(segment_duration / chunk_duration)
    if chunks_per_segment <= 0:
        raise ValueError("segments must include at least one chunk")
    hop_chunks = max(1, int(hop_chunks))

    total_chunks = feature_array.shape[0]
    if total_chunks < chunks_per_segment:
        return np.empty((0, chunks_per_segment, feature_array.shape[1]), dtype=feature_array.dtype)

    segments = []
    for start in range(0, total_chunks - chunks_per_segment + 1, hop_chunks):
        end = start + chunks_per_segment
        segments.append(feature_array[start:end])

    return np.stack(segments, axis=0)


In [334]:
all_segments = []
all_labels = []
for entry in dataset:
    feats = chunk_fft_features(entry['audio'], sr)
    segs = make_segments(feats)
    if len(segs) == 0:
        continue
    all_segments.append(segs)
    all_labels.append(np.full(len(segs), entry['label'], dtype=np.float32))
    print(f"{entry['name']}: segments={len(segs)}, segment_shape={segs.shape[1:]}")

if not all_segments:
    raise ValueError("No segments created from available wav files.")

data_final = np.concatenate(all_segments)
labels_final = np.concatenate(all_labels)

print("data_final", data_final.shape, "labels_final", labels_final.shape)


group_room_2_9_dec_presence: segments=237, segment_shape=(4, 207)
group_room_3_yazan_lab_9_dec_presence: segments=117, segment_shape=(4, 207)
group_room_4_9_dec_presence: segments=237, segment_shape=(4, 207)
group_room_5_9_dec_presence: segments=237, segment_shape=(4, 207)
group_room_9_dec_presence: segments=117, segment_shape=(4, 207)
test_presence: segments=7, segment_shape=(4, 207)
training-lax_no_presence: segments=117, segment_shape=(4, 207)
training-lax_presence: segments=117, segment_shape=(4, 207)
training2-lax_no_presence: segments=117, segment_shape=(4, 207)
training2-lax_presence: segments=117, segment_shape=(4, 207)
data_final (1420, 4, 207) labels_final (1420,)


In [335]:
train_data, validation_data, train_labels, validation_labels = train_test_split(
    data_final, labels_final, test_size=0.2, shuffle=True
)


In [336]:
train_data.shape


(1136, 4, 207)

In [337]:
# Reshape data for Conv2D (time, freq, channel)
train_data_reshaped = train_data[..., np.newaxis]
validation_data_reshaped = validation_data[..., np.newaxis]

input_shape = (train_data_reshaped.shape[1], train_data_reshaped.shape[2], 1)

model = K.models.Sequential()
model.add(K.layers.Conv2D(filters=16, kernel_size=(2, 4), strides=(1, 2), activation='relu', input_shape=input_shape))
model.add(K.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 2)))
model.add(K.layers.Conv2D(filters=8, kernel_size=(1, 4), strides=(1, 2), activation='relu'))
model.add(K.layers.MaxPool2D(pool_size=(1, 2), strides=(1, 2)))
model.add(K.layers.Flatten())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dense(1, activation='sigmoid'))

model.summary()


/Users/yazan.al-aswad/OtherCode/School/IOT/ML-for-iot-project-presence-detector/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 3, 102, 16)     │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 2, 51, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 2, 24, 8)       │           520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 2, 12, 8)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_23 (Flatten)            │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,081 (51.10 KB)

 Trainable params: 13,081 (51.10 KB)

 Non-trainable params: 0 (0.00 B)

In [338]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
checkpoint_cb = K.callbacks.ModelCheckpoint(
    filepath="best.weights.h5",
    monitor="val_accuracy",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)


In [339]:
history = model.fit(
    train_data_reshaped,
    train_labels,
    validation_data=(validation_data_reshaped, validation_labels),
    epochs=10,
    verbose=1,
    callbacks=[checkpoint_cb],
)

best_path = "best.weights.h5"
if Path(best_path).exists():
    model.load_weights(best_path)
    print(f"Loaded best weights from {best_path}")


Epoch 1/10
24/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8471 - loss: 0.2699   
Epoch 1: val_accuracy improved from None to 0.95423, saving model to best.weights.h5
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8829 - loss: 0.2166 - val_accuracy: 0.9542 - val_loss: 0.1662
Epoch 2/10
24/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9503 - loss: 0.1603 
Epoch 2: val_accuracy did not improve from 0.95423
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9569 - loss: 0.1383 - val_accuracy: 0.9542 - val_loss: 0.1136
Epoch 3/10
25/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9620 - loss: 0.1185 
Epoch 3: val_accuracy did not improve from 0.95423
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9674 - loss: 0.1025 - val_accuracy: 0.9472 - val_loss: 0.1095
Epoch 4/10
23/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9753 - loss: 0.0779 
Epoch 4: val_accuracy improved from 0.95423 to 0.97183, saving model to best.weights.h5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

In [340]:
print("run model");

run model


In [341]:
from tinymlgen import port

c_code = port(model, variable_name='seizure_model', pretty_print=True,optimize=False)
filename = 'arduino/net.h'
with open(filename,'w') as f: 
    f.write(c_code)

INFO:tensorflow:Assets written to: /var/folders/nf/hck60h0n5dl_8jn9fz3mlz080000gp/T/tmp58u3p8p8/assets


INFO:tensorflow:Assets written to: /var/folders/nf/hck60h0n5dl_8jn9fz3mlz080000gp/T/tmp58u3p8p8/assets


Saved artifact at '/var/folders/nf/hck60h0n5dl_8jn9fz3mlz080000gp/T/tmp58u3p8p8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4, 207, 1), dtype=tf.float32, name='keras_tensor_598')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  14619716176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14619716368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14619715984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14619715600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14619713488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14619714256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14619714064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  14619717712: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1765292134.610879 10127952 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1765292134.610898 10127952 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-09 15:55:34.611022: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/nf/hck60h0n5dl_8jn9fz3mlz080000gp/T/tmp58u3p8p8
2025-12-09 15:55:34.611493: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-12-09 15:55:34.611499: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/nf/hck60h0n5dl_8jn9fz3mlz080000gp/T/tmp58u3p8p8
2025-12-09 15:55:34.615507: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-12-09 15:55:34.637979: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/nf/hck60h0n5dl_8jn9fz3mlz080000gp/T/tmp58u3p8p8
2025-12-09 15:55:34.646016: I tensorflow/cc/saved_model/loader.c